In [1]:
# 필요한 라이브러리 불러오기
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score
import xgboost as xgb

# 데이터 로드
# data = pd.read_csv("your_data.csv")  # 데이터 파일 경로로 교체
# 이미 데이터프레임이 있다고 가정 (data)

# 1. 데이터 전처리
# 결측치 처리 (범주형 변수는 'Unknown'으로, 수치형 변수는 평균으로 대체)
categorical_columns = data.select_dtypes(include=['object']).columns
numeric_columns = data.select_dtypes(include=['int64', 'float64']).columns

# 범주형 변수 결측치 처리
data[categorical_columns] = data[categorical_columns].fillna('Unknown')

# 수치형 변수 결측치 처리
data[numeric_columns] = data[numeric_columns].fillna(data[numeric_columns].mean())

# 범주형 변수 인코딩 (One-Hot Encoding)
data = pd.get_dummies(data, columns=categorical_columns, drop_first=True)

# 타겟 변수와 독립 변수 설정
X = data.drop('Churn', axis=1)  # 타겟 변수 'Churn'을 제외한 나머지
y = data['Churn']  # 타겟 변수 (이탈 여부)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Gradient Boosting 모델 학습 (XGBoost)
# 모델 초기화 및 학습
model = xgb.XGBClassifier(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    random_state=42,
    use_label_encoder=False,
    eval_metric="logloss"
)

model.fit(X_train, y_train)

# 3. 모델 평가
# 예측
y_pred = model.predict(X_test)
y_pred_prob = model.predict_proba(X_test)[:, 1]

# 성능 지표 출력
print("Classification Report:")
print(classification_report(y_test, y_pred))

print("ROC-AUC Score:", roc_auc_score(y_test, y_pred_prob))
print("Accuracy Score:", accuracy_score(y_test, y_pred))

# 4. 변수 중요도 시각화
import matplotlib.pyplot as plt
xgb.plot_importance(model, max_num_features=10, importance_type="gain")
plt.title("Feature Importance")
plt.show()


ModuleNotFoundError: No module named 'xgboost'